In [1]:
# from utils import preprocess, train_test_split_data
from importlib import reload
import os
import utils
utils = reload(utils)

## Task 1: Data preparation

In [2]:
# import nltk, nltk.data

# nltk.download('punkt')

# tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
# with open('data/alice_in_wonderland.txt') as f:
#     text = f.read()
#     print('\n-----\n'.join(tokenizer.tokenize(text)))

In [3]:
# sentencepiece package look for sentence hence linebreak needs to be preserved

with open('data/alice_in_wonderland.txt') as f:
    text = f.read()
    prepro_text = utils.preprocess(text) # preprocessing text
    train, test = utils.train_test_split_data(prepro_text, test_size=0.2) # split the data


In [4]:
# saving the splitted corpus
with open('eng_text/train_eng.txt', 'w') as f:
    # f.write(str(train))
    f.write('\n'.join(train))

with open('eng_text/test_eng.txt', 'w') as f:
    # f.write(str(test)) 
    f.write('\n'.join(test))

### Comments/ideas
* split is not randomized
* more steps like lemmatization/stemming can be added in prepro
* there is not much preprocessing needs to be done
* may be some special characters/ punctuation can be removed

## Task 2: Subword Segmentation

### 1. Granularity: characters

In [5]:
# training model

## CHANGE PATH TO SHIFT .model & .vocab in models/sentencepiece

# total 72 different types of characters
# coverage changes for non-english

n_ch = 72

!cd models/sentencepiece/ \
  &&spm_train \
  --input='../../eng_text/train_eng.txt' \
  --model_prefix=en_s1_train \
  --vocab_size=$n_ch \
  --character_coverage=1.0 \
  --model_type=bpe
 

sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: ../../eng_text/train_eng.txt
  input_format: 
  model_prefix: en_s1_train
  model_type: BPE
  vocab_size: 95
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
}
normalizer_spec {
  name: nmt_nfkc
  add_dummy_prefix: 1
  remove_extra_whitespaces: 1
  

In [6]:
# segment the text (original)
!cd models/sentencepiece/ \
  &&spm_encode \
  --model=en_s1_train.model \
  --output_format=piece \
  < '../../eng_text/train_eng.txt' \
  > '../../eng_text/en_s1_train.txt'

In [7]:
# training model - test set

## CHANGE PATH TO SHIFT .model & .vocab in models/sentencepiece

# total 72 different types of characters
# coverage changes for non-english


!cd models/sentencepiece/ \
  &&spm_train \
  --input='../../eng_text/test_eng.txt' \
  --model_prefix=en_s1_test \
  --vocab_size=$n_ch \
  --character_coverage=1.0 \
  --model_type=bpe
 

sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: ../../eng_text/test_eng.txt
  input_format: 
  model_prefix: en_s1_test
  model_type: BPE
  vocab_size: 95
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
}
normalizer_spec {
  name: nmt_nfkc
  add_dummy_prefix: 1
  remove_extra_whitespaces: 1
  es

In [8]:
# segment the text (original) - test set

!cd models/sentencepiece/ \
  &&spm_encode \
  --model=en_s1_test.model \
  --output_format=piece \
  < '../../eng_text/test_eng.txt' \
  > '../../eng_text/en_s1_test.txt'

### 2. Granularity: subword units (smaller vocabulary)

In [9]:
# training model

# coverage changes for non-english
# fine-tune vocab_size in range of 100-800 for best performance

small_vocab = 450

!cd models/sentencepiece/ \
  &&spm_train \
  --input='../../eng_text/train_eng.txt' \
  --model_prefix=en_s2_train \
  --vocab_size=$small_vocab \
  --character_coverage=1.0 \
  --model_type=bpe
 

sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: ../../eng_text/train_eng.txt
  input_format: 
  model_prefix: en_s2_train
  model_type: BPE
  vocab_size: 450
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
}
normalizer_spec {
  name: nmt_nfkc
  add_dummy_prefix: 1
  remove_extra_whitespaces: 1
 

In [10]:
# segment the text (original) -train
!cd models/sentencepiece/ \
  &&spm_encode \
  --model=en_s2_train.model \
  --output_format=piece \
  < '../../eng_text/train_eng.txt' \
  > '../../eng_text/en_s2_train.txt'

In [11]:
# training model - test set

# coverage changes for non-english
# fine-tune vocab_size in range of 100-800 for best performance

!cd models/sentencepiece/ \
  &&spm_train \
  --input='../../eng_text/test_eng.txt' \
  --model_prefix=en_s2_test \
  --vocab_size=$small_vocab \
  --character_coverage=1.0 \
  --model_type=bpe
 

sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: ../../eng_text/test_eng.txt
  input_format: 
  model_prefix: en_s2_test
  model_type: BPE
  vocab_size: 450
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
}
normalizer_spec {
  name: nmt_nfkc
  add_dummy_prefix: 1
  remove_extra_whitespaces: 1
  e

In [12]:
# segment the text (original)
!cd models/sentencepiece/ \
  &&spm_encode \
  --model=en_s2_test.model \
  --output_format=piece \
  < '../../eng_text/test_eng.txt' \
  > '../../eng_text/en_s2_test.txt'

### 3. Granularity: subword units (larger vocabulary)

In [13]:
# training model

# coverage changes for non-english
# fine-tune vocab_size in range of 1500-3000 for best performance
# training model

# coverage changes for non-english
# fine-tune vocab_size in range of 100-800 for best performance

large_vocab = 2000

!cd models/sentencepiece/ \
  &&spm_train \
  --input='../../eng_text/train_eng.txt' \
  --model_prefix=en_s3_train \
  --vocab_size=$large_vocab \
  --character_coverage=1.0 \
  --model_type=bpe
 

sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: ../../eng_text/train_eng.txt
  input_format: 
  model_prefix: en_s3_train
  model_type: BPE
  vocab_size: 2000
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
}
normalizer_spec {
  name: nmt_nfkc
  add_dummy_prefix: 1
  remove_extra_whitespaces: 1


In [14]:
# segment the text (original)
!cd models/sentencepiece/ \
  &&spm_encode \
  --model=en_s3_train.model \
  --output_format=piece \
  < '../../eng_text/train_eng.txt' \
  > '../../eng_text/en_s3_train.txt'

In [15]:
# training model - test set

# coverage changes for non-english
# fine-tune vocab_size in range of 1500-3000 for best performance
# training model

# coverage changes for non-english
# fine-tune vocab_size in range of 100-800 for best performance

!cd models/sentencepiece/ \
  &&spm_train \
  --input='../../eng_text/test_eng.txt' \
  --model_prefix=en_s3_test \
  --vocab_size=$large_vocab \
  --character_coverage=1.0 \
  --model_type=bpe
 

sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: ../../eng_text/test_eng.txt
  input_format: 
  model_prefix: en_s3_test
  model_type: BPE
  vocab_size: 2000
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
}
normalizer_spec {
  name: nmt_nfkc
  add_dummy_prefix: 1
  remove_extra_whitespaces: 1
  

In [16]:
# segment the text (original)
!cd models/sentencepiece/ \
  &&spm_encode \
  --model=en_s3_test.model \
  --output_format=piece \
  < '../../eng_text/test_eng.txt' \
  > '../../eng_text/en_s3_test.txt'

## Observation
* Character - Almost every single character is segmented
* Subword unit (smaller vacob) - The length of segmented subword is longer and many words are also considered as subwords
* Subword unit (larger vocab) - Here, subwords are longer. Many words are themself segmented into single subwords. It can also be seen consistently that orignally longer words are broken into two or more segments.  


## Task 3: LM Training

In [17]:
# Training baseline LM (eng_s1)

# At what setting of rnnlm, we need the get the PP of baseline? 
# what is the use of class size

!cd models/rnnlm \
    && ../../rnnlm/rnnlm \
    -train '../../eng_text/en_s1_train.txt' \
    -valid '../../eng_text/en_s1_test.txt' \
    -rnnlm en_s1 \
      -hidden 40 \
      -rand-seed 1 \
      -debug 2 \
      -bptt 3 \
      -class $n_ch

debug mode: 2
train file: ../../eng_text/en_s1_train.txt
valid file: ../../eng_text/en_s1_test.txt
class size: 95
Hidden layer size: 40
BPTT: 3
Rand seed: 1
rnnlm file: en_s1
Starting training using file ../../eng_text/en_s1_train.txt
Restoring network from file to continue training...
Iter:   3	Alpha: 0.025000	   TRAIN entropy: 4.9836    Words/sec: 88832.1   VALID entropy: 3.8072
Iter:   4	Alpha: 0.012500	   TRAIN entropy: 3.2937    Words/sec: 90785.7   VALID entropy: 3.7471
Iter:   5	Alpha: 0.006250	   TRAIN entropy: 3.2628    Words/sec: 112106.8   VALID entropy: 3.7099
Iter:   6	Alpha: 0.003125	   TRAIN entropy: 3.2470    Words/sec: 98864.9   VALID entropy: 3.6900
Iter:   7	Alpha: 0.001563	   TRAIN entropy: 3.2386    Words/sec: 97880.2   VALID entropy: 3.6813


In [18]:
# Training LM on subword units (smaller vocab - eng_s2)

# At what setting of rnnlm, we need the get the PP of baseline? 
# what is the use of class size
# How to increase no. of iterations?

#incr hidden -> better pp
#incr bptt -> worse pp
# if class < vocabsize -> worse pp
#incr class -> better pp

## May be need to simplify the corpus (in preprocessing)

      # -bptt-block 1 \
!cd models/rnnlm/ \
    && ../../rnnlm/rnnlm \
    -train '../../eng_text/en_s2_train.txt' \
    -valid '../../eng_text/en_s2_test.txt' \
    -rnnlm en_s2 \
      -hidden 100 \
      -rand-seed 1 \
      -debug 2 \
      -bptt 0 \
      -class $small_vocab

debug mode: 2
train file: ../../eng_text/en_s2_train.txt
valid file: ../../eng_text/en_s2_test.txt
class size: 450
Hidden layer size: 100
BPTT: 0
Rand seed: 1
rnnlm file: en_s2
Starting training using file ../../eng_text/en_s2_train.txt
Restoring network from file to continue training...
Iter:  12	Alpha: 0.001563	   TRAIN entropy: 5.8887    Words/sec: 11469.3   VALID entropy: 5.8379


In [19]:
# Training LM on subword units (larger vocab - eng_s3)

# At what setting of rnnlm, we need the get the PP of baseline? 
# what is the use of class size
# How to increase no. of iterations?

#incr hidden -> better pp
#incr bptt -> worse pp
# if class < vocabsize -> worse pp
#incr class -> better pp

## May be need to simplify the corpus (in preprocessing)

      # -bptt-block 1 \
!cd models/rnnlm/ \
    && ../../rnnlm/rnnlm \
    -train '../../eng_text/en_s3_train.txt' \
    -valid '../../eng_text/en_s3_test.txt' \
    -rnnlm en_s3 \
      -hidden 120 \
      -rand-seed 1 \
      -debug 2 \
      -bptt 0 \
      -class $large_vocab

debug mode: 2
train file: ../../eng_text/en_s3_train.txt
valid file: ../../eng_text/en_s3_test.txt
class size: 2000
Hidden layer size: 120
BPTT: 0
Rand seed: 1
rnnlm file: en_s3
Starting training using file ../../eng_text/en_s3_train.txt
Restoring network from file to continue training...
Iter:  13	Alpha: 0.000781	   TRAIN entropy: 7.0128    Words/sec: 2016.5   VALID entropy: 5.9636


## Task 4: Text Generation

In [20]:
## 1. Character level granularity

!cd models/rnnlm/ \
    && for i in 10 100 1000 10000 100000 1000000 10000000; do \
            ../../rnnlm/rnnlm \
            -rnnlm  en_s1\
            -gen $i \
            -debug 0 \
            >> "../../eng_text/gen_en_s1/${i}.txt"; \
       done

In [21]:
## 2. subword units (smaller vocab - en_s2)

!cd models/rnnlm \
    && for i in 10 100 1000 10000 100000 1000000 10000000; do \
            ../../rnnlm/rnnlm \
            -rnnlm en_s2 \
            -gen $i \
            -debug 0 \
            >> "../../eng_text/gen_en_s2/${i}.txt"; \
       done

In [ ]:
## 3. subword units (larger vocab - en_s3)

!cd models/rnnlm \
    && for i in 10 100 1000 10000 100000 1000000 10000000; do \
            ../../rnnlm/rnnlm \
            -rnnlm en_s3 \
            -gen $i \
            -debug 0 \
            >> "../../eng_text/gen_en_s3/${i}.txt"; \
       done

^C


In [ ]:
# back to original (human readable form) from subword units 

!cd models/sentencepiece/ \
  &&spm_decode \
  --model=en_s1_train.model \
  --input_format=piece \
  < "../../eng_text/gen_en_s1/100.txt" \
  > "../../eng_text/gen_en_s1/decod_100.txt"

In [ ]:
# back to original (human readable form) from subword units 

!cd models/sentencepiece/ \
  &&spm_decode \
  --model=en_s2_train.model \
  --input_format=piece \
  < "../../eng_text/gen_en_s2/100.txt" \
  > "../../eng_text/gen_en_s2/decod_100.txt"

In [ ]:
# back to original (human readable form) from subword units 

!cd models/sentencepiece/ \
  &&spm_decode \
  --model=en_s3_train.model \
  --input_format=piece \
  < "../../eng_text/gen_en_s3/100.txt" \
  > "../../eng_text/gen_en_s3/decod_100.txt"

### Observation - quality of generated text
*
*
*